Piping Chains and the RunnablePassthrough Class

In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [5]:
RunnablePassthrough().invoke([1,2,3]) # kind of like an identity function for langchain

[1, 2, 3]

In [8]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')


chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [9]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [10]:
chat = ChatOpenAI(
    model_name = 'gpt-4',
    model_kwargs = {'seed':365},
    temperature = 0,
    max_tokens = 100
)

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [11]:
string_parser = StrOutputParser()

In [17]:
chain_tools = chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()}
chain_strategy = chat_template_strategy | chat | string_parser


In [18]:
print(chain_tools.invoke({'job title': 'data scientist'}))

{'tools': '1. Python\n2. R Programming\n3. SQL\n4. Tableau\n5. Hadoop'}


In [16]:
print(chain_strategy.invoke({'tools': '''
1. Python
2. R Programming Language
3. SQL (Structured Query Language)
4. Tableau
5. Apache Hadoop
'''}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more complex topics like classes and objects. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming Language: Start with the basics of R, such as


In [19]:
chain_combined = chain_tools | chain_strategy

In [20]:
print(chain_combined.invoke({'job title': 'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more complex topics like classes and objects. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming Language: Start with the basics of R, such as


In [21]:
chain_long = ( chat_template_tools | chat | string_parser | {'tools': RunnablePassthrough()}| 
                chat_template_strategy | chat | string_parser )